In [1]:
import gymnasium as gym
import math
import random
import matplotlib
import matplotlib.pyplot as plt
from collections import namedtuple, deque
from itertools import count
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.distributions import Categorical,normal

# env = gym.make("CartPole-v1")
env = gym.make("Hopper-v5")


# set up matplotlib
is_ipython = 'inline' in matplotlib.get_backend()
if is_ipython:
    from IPython import display

plt.ion()

# if GPU is to be used
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
SavedAction = namedtuple('SavedAction', ['log_prob', 'value'])

In [2]:
gamma = 0.99
eps = np.finfo(np.float32).eps.item()

In [3]:
class Actor(nn.Module):

    def __init__(self):
        super(Actor, self).__init__()
        self.linear = nn.Linear(11, 128)

        self.action_mean = nn.Linear(128, 3)  # 3 actions, mean for each action
        self.action_std = nn.Linear(128, 3)  # std for each action
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()
        # self.value_head = nn.Linear(128, 1)

    def forward(self, x):

        x = self.linear(x)

        # actor: choses action to take from state s_t
        # by returning probability of each action
        action_prob =self.relu(self.action_mean(x))
        action_std = self.softplus(self.action_std(x))+eps

        # return values for both actor and critic as a tuple of 2 values:
        # 1. a list with the probability of each action over the action space
        # 2. the value from state s_t
        return action_prob, action_std
    
class Critic(nn.Module):
    def __init__(self):
        super(Critic, self).__init__()
        self.linear = nn.Linear(11, 128)
        self.value = nn.Linear(128, 1)

    def forward(self, x):
        x = self.linear(x)
        state_values = self.value(x)
        return state_values

In [4]:
actor = Actor()
critic = Critic()
actor_optimizer = optim.Adam(actor.parameters(), lr=1e-2)
critic_optimizer = optim.Adam(critic.parameters(), lr=1e-2)
# hyperparameters


In [5]:
def get_action(state):
    state = torch.from_numpy(state).float()
    probs_mean, probs_std= actor(state)
    actions=[]
    act_log_prob = []
    # create a categorical distribution over the list of probabilities of actions
    for i in range(len(probs_mean)):
        m = normal.Normal(probs_mean[i],probs_std[i])  # using normal distribution for sampling

    # and sample an action using the distribution
        action = m.sample()
        actions.append(action)
        act_log_prob.append(m.log_prob(action))
    actions = torch.tensor(actions)
    logp = torch.sum(torch.stack(act_log_prob),dim=0)
    return actions.numpy(),logp

def get_value(state):
    state = torch.from_numpy(state).float()
    state_value = critic(state)
    return state_value

In [6]:
def compute_loss(rewards,acts,values,logp):
    policy_losses = []
    value_losses = [] # list to save critic (value) loss
    returns=[]
    R = 0
    for r in rewards[::-1]:
        # calculate the discounted value
        R = r + gamma * R
        returns.insert(0, R)
    returns = torch.tensor(returns)
    returns = (returns - returns.mean()) / (returns.std() + eps)
    for log_prob, value, R in zip(logp, values,rewards):
        advantage = R - value

        # calculate actor (policy) loss
        policy_losses.append(-log_prob * advantage.detach())
        value_losses.append(F.smooth_l1_loss(value, torch.tensor([R])))


        # sum up all the values of policy_losses and value_losses
    policyloss = torch.stack(policy_losses).sum()
    criticloss = torch.stack(value_losses).sum()
    return policyloss, criticloss

In [7]:
num_episodes = 10000
log_interval = 10
render=False
# run infinitely many episodes
for i_episode in range(num_episodes):

    # reset environment and episode reward
    state, _ = env.reset()
    episode_rewards = []
    episode_acts = []
    episode_values = []
    episode_logp = []
    # for each episode, only run 9999 steps so that we don't
    # infinite loop while learning
    for t in range(1, 10000):

        # select action from policy
        action,logp = get_action(state)
        # print("action",action,"logp",logp)
        value = get_value(state)
        # take the action
        state, reward, terminated, truncated, _ = env.step(action)

        episode_rewards.append(reward)
        episode_acts.append(action)
        episode_values.append(value)
        episode_logp.append(logp)
        if terminated or truncated:
            break

    # perform backprop
    # reset gradients
    actor_optimizer.zero_grad()
    critic_optimizer.zero_grad()
    actor_loss,critic_loss = compute_loss(episode_rewards, episode_acts, episode_values,episode_logp)
    # print("actor_loss",actor_loss,"critic_loss",critic_loss)
    actor_loss.backward()
    critic_loss.backward()

    actor_optimizer.step()
    critic_optimizer.step()
    # log results
    if i_episode % log_interval == 0:
        print('Episode {}\tLast reward: {:.2f}'.format(
                i_episode, np.sum(episode_rewards)))




Episode 0	Last reward: 81.47
Episode 10	Last reward: 204.08
Episode 20	Last reward: 111.62
Episode 30	Last reward: 35.83
Episode 40	Last reward: 14.75
Episode 50	Last reward: 17.82
Episode 60	Last reward: 40.11
Episode 70	Last reward: 169.41
Episode 80	Last reward: 44.26
Episode 90	Last reward: 144.39
Episode 100	Last reward: 242.86
Episode 110	Last reward: 163.77
Episode 120	Last reward: 53.83
Episode 130	Last reward: 14.70
Episode 140	Last reward: 152.97
Episode 150	Last reward: 9.97
Episode 160	Last reward: 20.05
Episode 170	Last reward: 76.10
Episode 180	Last reward: 45.80
Episode 190	Last reward: 12.20
Episode 200	Last reward: 32.55
Episode 210	Last reward: 120.28
Episode 220	Last reward: 109.24
Episode 230	Last reward: 54.33
Episode 240	Last reward: 21.57
Episode 250	Last reward: 22.13
Episode 260	Last reward: 77.20
Episode 270	Last reward: 14.65
Episode 280	Last reward: 35.68
Episode 290	Last reward: 41.95
Episode 300	Last reward: 22.05
Episode 310	Last reward: 16.39
Episode 320

In [8]:
# def train_one_epoch():
#     # make some empty lists for logging.
#     batch_states = []          # for observations
#     batch_acts = []         # for actions
#     batch_weights = []      # for R(tau) weighting in policy gradient
#     batch_rets = []         # for measuring episode returns
#     batch_lens = []         # for measuring episode lengths
#     batch_size = 10000      # number of steps to take in each epoch
#     # reset episode-specific variables
#     states = env.reset()       # first obs comes from starting distribution
#     done = False            # signal from environment that episode is over
#     epoch_rewards = []            # list for rewards accrued throughout ep


#     # collect experience by acting in the environment with current policy
#     while True:

#         # save states
#         batch_states.append(states.copy())

#         # act in the environment
#         act = get_action(torch.as_tensor(states, dtype=torch.float32))
#         states, reward, terminated, truncated,_ = env.step(act)

#         # save action, reward
#         batch_acts.append(act)
#         epoch_rewards.append(reward)

#         if terminated or truncated:
#             # if episode is over, record info about episode
#             epoch_return, epoch_len = sum(epoch_rewards), len(epoch_rewards)
#             batch_rets.append(ep_ret)
#             batch_lens.append(ep_len)

#             # the weight for each logprob(a|s) is R(tau)
#             batch_weights += [ep_ret] * ep_len

#             # reset episode-specific variables
#             obs, done, ep_rews = env.reset(), False, []


#             # end experience loop if we have enough of it
#             if len(batch_obs) > batch_size:
#                 break

#     # take a single policy gradient update step
#     actor_optimizer.zero_grad()
#     batch_loss = compute_loss(obs=torch.as_tensor(batch_obs, dtype=torch.float32),
#                               act=torch.as_tensor(batch_acts, dtype=torch.int32),
#                               weights=torch.as_tensor(batch_weights, dtype=torch.float32)
#                               )
#     batch_loss.backward()
#     critic_optimizer.step()
#     return batch_loss, batch_rets, batch_lens